<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np

In [2]:
import statsmodels.api as sm

In [13]:
data = np.random.normal(size=(100, 3))

In [118]:
import numpy as np
import pandas
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.base.datetools import dates_from_str

mdata = sm.datasets.macrodata.load_pandas().data
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)
mdata = mdata[['realgdp','realcons','realinv']]
mdata.index = pandas.DatetimeIndex(quarterly)
data = np.log(mdata).diff().dropna()

In [120]:
data.to_csv('statsmodels_macrodata_processed.csv')

In [121]:
data

,realgdp,realcons,realinv
1959-06-30,0.024942,0.015286,0.080213
1959-09-30,-0.001193,0.010386,-0.072131
1959-12-31,0.003495,0.001084,0.034425
1960-03-31,0.022190,0.009534,0.102664
1960-06-30,-0.004685,0.012572,-0.106694
...,...,...,...
2008-09-30,-0.006781,-0.008948,-0.017836
2008-12-31,-0.013805,-0.007843,-0.069165
2009-03-31,-0.016612,0.001511,-0.175598
2009-06-30,-0.001851,-0.002196,-0.067561


In [308]:
from warnings import catch_warnings, simplefilter

with catch_warnings():
    simplefilter('ignore')
    mod = sm.tsa.VARMAX(data, order=(3, 5), trend='n')

In [348]:
cumcounts = [None] + np.cumsum(list(mod.parameters.values())).tolist()
param_slices = [slice(a, b) for a, b in zip(cumcounts[:-1], cumcounts[1:])]
param_lists = [trend, ar, ma, reg, state_cov, obs_cov] = [mod.param_names[idx] for idx in param_slices]
param_d = {k: np.random.normal() ** 2 for param_list in param_lists for k in param_list}

In [349]:
np.array([param_d[var] for var in ma])

array([9.96574640e-02, 1.65345932e-01, 1.59660576e+00, 5.28132552e-01,
       4.81236515e-02, 8.97510827e-01, 2.98831375e-01, 2.26580733e+00,
       8.78934614e-01, 2.06931456e+00, 6.66088212e-01, 2.39139619e-02,
       7.46655240e-01, 1.72449685e-02, 4.15539812e+00, 1.18947918e-01,
       2.47732947e-01, 4.52037954e-02, 2.26644073e+00, 1.93359145e-01,
       9.91466784e-01, 8.92611496e-01, 4.22432820e-01, 8.43731845e-02,
       1.87137334e-02, 5.26878794e-01, 1.05874149e+00, 2.74374266e+00,
       5.85570112e-01, 1.32070810e+00, 4.61457953e-05, 1.52774337e+00,
       1.10589872e-01, 3.56659582e-01, 4.89497037e-02, 4.23597254e-01,
       2.53001406e-03, 6.58794455e-01, 9.07563817e-02, 1.09596955e+00,
       2.64076674e-01, 1.24584788e-02, 8.78265112e-01, 1.29984908e+00,
       5.24482083e-01])

In [345]:
mod.ssm['state_cov']

array([[0.234856  , 0.14561773, 0.00419308],
       [0.14561773, 0.12337399, 0.08554384],
       [0.00419308, 0.08554384, 0.27491135]])

In [346]:
param_d

{'L1.realgdp.realgdp': 0.09446069431900957,
 'L1.realcons.realgdp': 1.4859065314947975,
 'L1.realinv.realgdp': 0.005348710946751118,
 'L2.realgdp.realgdp': 0.025072232577998517,
 'L2.realcons.realgdp': 0.047359095779670256,
 'L2.realinv.realgdp': 6.89583145267583,
 'L3.realgdp.realgdp': 0.24306175937297794,
 'L3.realcons.realgdp': 0.10014137242563821,
 'L3.realinv.realgdp': 0.06927006075682304,
 'L1.realgdp.realcons': 4.827021081902981,
 'L1.realcons.realcons': 0.5491378338194867,
 'L1.realinv.realcons': 0.665593733095405,
 'L2.realgdp.realcons': 2.0560204126622934,
 'L2.realcons.realcons': 2.159554647014486,
 'L2.realinv.realcons': 6.614781342120009,
 'L3.realgdp.realcons': 0.009029353720525763,
 'L3.realcons.realcons': 0.018001145931244713,
 'L3.realinv.realcons': 0.11107201327279365,
 'L1.realgdp.realinv': 0.1994099995721026,
 'L1.realcons.realinv': 0.8958539993549192,
 'L1.realinv.realinv': 2.815427036143525,
 'L2.realgdp.realinv': 1.5206083712256064,
 'L2.realcons.realinv': 0.1916

In [304]:
slice(-k_obs * q, -k_obs * (q-1))

slice(-15, -12, None)

In [305]:
end = -k_obs * (q-1) if q > 1 else None
mod.ssm['selection', slice(-k_obs * q, end), :]

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [113]:
k_obs, p = mod.k_endog, mod.k_ar
with np.printoptions(linewidth=1000, precision=3, suppress=True):
    print(mod.ssm[('transition', ) + (slice(0, k_obs), slice(k_obs * p, None))])

[]


In [114]:
with np.printoptions(linewidth=1000, precision=3, suppress=True):
    print(mod.ssm['selection'])

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [116]:
k_states = mod.k_states
R = np.zeros((k_states, k_obs))

R[slice(0, k_obs), slice(0, None)] = np.eye(k_obs)
R[np.arange(k_obs * p, k_obs * (p + 1)), np.arange(0, k_obs)] = 1

IndexError: index 9 is out of bounds for axis 0 with size 9

In [122]:
mod.ssm['state_cov']

array([[ 0.72381047, -0.06570529, -0.13207411],
       [-0.06570529,  0.7160295 ,  0.0789942 ],
       [-0.13207411,  0.0789942 ,  0.78839058]])

In [111]:
np.c_[R, mod.ssm['selection']]

array([[1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [82]:
mod.ssm[('transition', slice(k_obs, None), slice(0, k_obs * p))]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [89]:
eye = np.eye(k_obs * p)
eye[-k_obs:] = 0

In [90]:
eye

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [303]:
k_obs, p, q = mod.k_endog, mod.k_ar, mod.k_ma
with np.printoptions(linewidth=1000, precision=1, suppress=True):
    print(mod.ssm['selection'])

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [338]:
np.array(ar).reshape((3, 9))

array([['L1.realgdp.realgdp', 'L1.realcons.realgdp',
        'L1.realinv.realgdp', 'L2.realgdp.realgdp',
        'L2.realcons.realgdp', 'L2.realinv.realgdp',
        'L3.realgdp.realgdp', 'L3.realcons.realgdp',
        'L3.realinv.realgdp'],
       ['L1.realgdp.realcons', 'L1.realcons.realcons',
        'L1.realinv.realcons', 'L2.realgdp.realcons',
        'L2.realcons.realcons', 'L2.realinv.realcons',
        'L3.realgdp.realcons', 'L3.realcons.realcons',
        'L3.realinv.realcons'],
       ['L1.realgdp.realinv', 'L1.realcons.realinv',
        'L1.realinv.realinv', 'L2.realgdp.realinv',
        'L2.realcons.realinv', 'L2.realinv.realinv',
        'L3.realgdp.realinv', 'L3.realcons.realinv',
        'L3.realinv.realinv']], dtype='<U20')

In [334]:
obs_cov

[]

In [328]:
param_slices

[slice(None, 0, None),
 slice(0, 27, None),
 slice(27, 72, None),
 slice(72, 72, None),
 slice(72, 78, None),
 slice(78, 78, None)]

In [327]:
cumcounts

[None, 0, 27, 72, 72, 78, 78]

In [319]:
param_slices

[slice(0, 27, None),
 slice(27, 72, None),
 slice(72, 72, None),
 slice(72, 78, None),
 slice(78, 78, None),
 slice(78, None, None)]